In [76]:
import os
import shutil
import glob
import csv

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm
from jcopdl.callback import Callback
from pathlib import Path
from PIL import Image

version = 1
data_dir = f"data/one_vs_all_data_v{version}"

In [77]:
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [78]:
batch_size = 32

def get_one_vs_all_dataloader(pothole_type):
    train_set = datasets.ImageFolder(f"{data_dir}/{pothole_type}/train", transform=train_transform)
    val_set = datasets.ImageFolder(f"{data_dir}/{pothole_type}/valid", transform=test_transform)
    test_set = datasets.ImageFolder(f"{data_dir}/{pothole_type}/test", transform=test_transform)
    
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valloader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
    testloader = DataLoader(test_set, shuffle=True)
    
    return train_set, val_set, test_set, trainloader, valloader, testloader

In [79]:
trainloaders, valloaders, testloaders = {}, {}, {}
trainsets, valsets, testsets = {}, {}, {}

for pothole_type in ["mild", "moderate", "severe"]:
    train_set, val_set, test_set, trainloader, valloader, testloader = get_one_vs_all_dataloader(pothole_type)
    trainloaders[pothole_type] = trainloader
    valloaders[pothole_type] = valloader
    testloaders[pothole_type] = testloader
    trainsets[pothole_type] = train_set
    valsets[pothole_type] = val_set
    testsets[pothole_type] = test_set

In [80]:
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == 'train':
        model.train()
    elif mode == 'val':
        model.eval()

    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)

        if mode == 'train':
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()   

        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
        
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [82]:
def train_model(pothole_type):
    model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
    model.classifier[1] = torch.nn.Linear(model.last_channel, 2)
    
    train_set = trainsets[pothole_type]
    trainloader = trainloaders[pothole_type]
    config = {
        'batch_size': batch_size,
        'output_size': len(train_set.classes)
    }

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = torch.nn.NLLLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    callback = Callback(model, config, early_stop_patience=5, outdir='model')
    valloader = valloaders[pothole_type]
    
    while True:
        train_cost, train_score = loop_fn('train', train_set, trainloader, model, criterion, optimizer, device)
        with torch.no_grad():
            test_cost, test_score = loop_fn('val', val_set, valloader, model, criterion, optimizer, device)

        # Logging
        callback.log("train_cost", train_cost)
        callback.log("test_cost", test_cost)
        callback.log("train_score", train_score)
        callback.log("test_score", test_score)

        # Early Stopping
        if callback.early_stopping("maximize", monitor='test_score'):
            callback.add_plot(["train_cost", "test_cost"], scale="semilogy")
            callback.add_plot(["train_score", "test_score"], scale="semilogy")
            break
            
    return model

In [83]:
mild_model = train_model("mild")

Using cache found in /Users/yangjunhui/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/yangjunhui/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/yangjunhui/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Val: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.25it/s]


,train_cost,test_cost,train_score,test_score
epoch,,,,
1,-0.074172,-0.004152,0.532,0.53
2,-0.574324,-0.390456,0.668,0.63
3,-1.152925,-0.961766,0.794,0.74
4,-1.892242,-1.668226,0.816,0.75
5,-2.773800,-2.443144,0.826,0.74
6,-3.798905,-3.375942,0.831,0.72
7,-4.808210,-4.026488,0.828,0.74
8,-5.696913,-4.716081,0.836,0.74
9,-6.678553,-5.095028,0.859,0.76


Val: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.32it/s]


==> Execute Early Stopping at epoch: 19 | Best test_score: 0.8200
==> Best model is saved at model


In [84]:
moderate_model = train_model("moderate")

Using cache found in /Users/yangjunhui/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/yangjunhui/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/yangjunhui/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Val: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.25it/s]


,train_cost,test_cost,train_score,test_score
epoch,,,,
1,-0.437750,-0.645914,0.597,0.62
2,-0.819242,-1.001706,0.701,0.66
3,-1.218229,-1.400477,0.726,0.66
4,-1.720174,-1.792012,0.722,0.65
5,-2.308757,-2.445983,0.725,0.67
6,-3.016385,-3.085764,0.734,0.67
7,-3.846679,-3.690083,0.757,0.68
8,-4.742393,-4.456609,0.776,0.69
9,-5.687573,-5.185782,0.781,0.69


Val: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.40it/s]


==> Execute Early Stopping at epoch: 24 | Best test_score: 0.7400
==> Best model is saved at model


In [85]:
severe_model = train_model("severe")

Using cache found in /Users/yangjunhui/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/yangjunhui/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/yangjunhui/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Val: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.19it/s]


,train_cost,test_cost,train_score,test_score
epoch,,,,
1,0.167182,0.051205,0.528,0.59
2,-0.290618,-0.281434,0.684,0.65
3,-0.745630,-0.748886,0.755,0.68
4,-1.394814,-1.350903,0.794,0.69
5,-2.193448,-1.882032,0.819,0.73
6,-3.128183,-2.728799,0.822,0.75
7,-4.154732,-3.019528,0.829,0.77
8,-5.180964,-3.421646,0.852,0.77
9,-6.088742,-4.224546,0.858,0.76


Val: 100%|████████████████████████████████████████| 4/4 [00:01<00:00,  2.36it/s]


==> Execute Early Stopping at epoch: 12 | Best test_score: 0.7700
==> Best model is saved at model


In [102]:
import shutil
from pathlib import Path
from PIL import Image

def classify_image(source_dir, image_name, dest_dir):
    path = os.path.join(source_dir, image_name)
    image = Image.open(Path(path))
    input = test_transform(image)
    input = input.view(1, 3, 128, 128)

    mild_output = mild_model(input)
    moderate_output = moderate_model(input)
    severe_output = severe_model(input)
    
    mild_prob = torch.nn.functional.softmax(mild_output[0], dim=0)[0]
    moderate_prob = torch.nn.functional.softmax(moderate_output[0], dim=0)[0]
    severe_prob = torch.nn.functional.softmax(severe_output[0], dim=0)[0]
    max_prob = max(mild_prob, moderate_prob, severe_prob)
    
    mild_threshold = 0
    severe_threshold = 0
    if (max_prob == mild_prob and mild_prob > mild_threshold):
        dest = f"{dest_dir}/mild/"
    elif (max_prob == severe_prob and severe_prob > severe_threshold):
        dest = f"{dest_dir}/severe/"
    else:
        dest = f"{dest_dir}/moderate/"
    dest += image_name
        
    shutil.copyfile(path, dest)

In [104]:
scene_num = 1
source_dir = f"../potholes_evaluation/scene{scene_num}/imgs"
dest_dir = f"../potholes_evaluation/scene{scene_num}/results_one_vs_all_v4"

os.makedirs(f'{dest_dir}/mild')
os.makedirs(f'{dest_dir}/moderate')
os.makedirs(f'{dest_dir}/severe')

pathlist = Path(source_dir).glob('**/*.png')
for path in tqdm(pathlist):
    image_name = str(path).split("/")[-1]
    classify_image(source_dir, image_name, dest_dir)
    
f = open(f'{dest_dir}/results.csv', 'w')
writer = csv.writer(f)
writer.writerow(["image", "type"])

images = os.listdir(source_dir)
mild_images = os.listdir(f"{dest_dir}/mild")
moderate_images = os.listdir(f"{dest_dir}/moderate")
severe_images = os.listdir(f"{dest_dir}/severe")

for image in images:
    if image in mild_images:
        writer.writerow([image, "mild"])
    elif image in moderate_images:
        writer.writerow([image, "moderate"])
    else:
        writer.writerow([image, "severe"])

1190it [01:43, 11.47it/s]


In [100]:
def evaluate(image_path, pothole_type):
    image = Image.open(Path(image_path))
    input = test_transform(image)
    input = input.view(1, 3, 128, 128)

    mild_output = mild_model(input)
    moderate_output = moderate_model(input)
    severe_output = severe_model(input)
    
    mild_prob = torch.nn.functional.softmax(mild_output[0], dim=0)[0]
    moderate_prob = torch.nn.functional.softmax(moderate_output[0], dim=0)[0]
    severe_prob = torch.nn.functional.softmax(severe_output[0], dim=0)[0]
    max_prob = max(mild_prob, moderate_prob, severe_prob)
    
    mild_threshold = 0
    severe_threshold = 0
    if (max_prob == mild_prob and mild_prob > mild_threshold):
        res = "mild"
    elif (max_prob == severe_prob and severe_prob > severe_threshold):
        res = "severe"
    else:
        res = "moderate"
        
    if res == pothole_type:
        return 1
    return 0

In [101]:
correct = 0
total = 0

for pothole_type in ["mild", "moderate", "severe"]:
    test_dir = f"./data/one_vs_all_data_v1/{pothole_type}/test/0"
    
    pathlist = Path(test_dir).glob('**/*.jpg')
    for path in tqdm(pathlist):
        correct += evaluate(path, pothole_type)
        total += 1

correct/total

17it [00:01, 12.41it/s]
16it [00:01, 13.16it/s]
9it [00:00, 13.12it/s]


0.6428571428571429